In [5]:
import pandas as pd
import requests
from pprint import pprint
import requests
import time
import json

def drugconsumption(user_input, user_key):

    # credentials
    key = user_key
    Cookie = "38771cce9ac0f974522b34af498bfe66=56e404caec9315398b6da15a1e47abfc"

    url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/drugproduct?lang=en&type=json&status=1&user-key="
    url += key

    headers = {
      'user-key': key,
      'Cookie': Cookie
    }
    
    try:
    
        response = requests.request("GET", url, headers=headers)
        data = response.json()
    
    except:
        
        print("The website did not respond, please try again!")
        return
    
    drug_codes = []
    brand_names = []
    code_brands = []

    n = len(data)
    # print( (data[7]['drug_code'], data[7]['brand_name'] ) )
    for i in range(n):
        drug_codes.append(data[i]['drug_code'])
        brand_names.append(data[i]['brand_name'])
        code_brands.append((data[i]['drug_code'],data[i]['brand_name']))

    df = pd.DataFrame({'drug_code': drug_codes, 'brand_name': brand_names})

    #############################################################################################################
    # brand_names # used to check the branch names list
    #############################################################################################################


    # filtering drug ids and brand names

    #############################################################################################################
    # different user inputs
    # user_input = "AVONEX"
    # user_input = "ACETAMINOPHEN 325MG"
    # user_input = "CLONAZEPAM"
    # user_input = "BROMAZEPAM"
    # user_input = "LIP BALM SPF 15"
    # user_input = "EPHEDRINE SULFATE INJECTION USP"
    #############################################################################################################

    user_input = user_input.lower()
    filters = []

    # finding matching brand names
    for brand in brand_names:
        if user_input in brand.lower():
    #         print(brand_name.index(brand))
            filters.append(brand)

    filters = list(set(filters)) # removing duplicates

    # finding corresponding drug ids
    filters_ids = []
    for item in filters:
        k = brand_names.index(item)
        filters_ids.append(drug_codes[k])

    #############################################################################################################
    # to check drug ids and brand names that are filtered    
    # pprint(filters)
    # pprint(filters_ids)
    #############################################################################################################
    
    # data validation
    
    if not filters:
        print("There is no match for that brand name.")
        return
    else:
        
        # route of administration and form of the drug 

        # some empty declarations
        form_list = []
        route_list = []
        brand_namelist=[]

        # creating urls
        route_url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/route?lang=en&type=json&id="
        form_url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/form?lang=en&type=json&id="

        # inputting drug ids and retrieving results
        for drug_id in filters_ids:

            # creating url
            route_url += (str(drug_id) + "&active=yes")
            form_url += (str(drug_id) + "&active=yes")

            # getting responses
            route_response = requests.request("GET", route_url, headers=headers)
            time.sleep(1)
            form_response = requests.request("GET", form_url, headers=headers)

            # converting to list of dictionaries
            route_data = route_response.json()
            form_data = form_response.json()
            # extracting required information

            form_list.append(form_data[0]['pharmaceutical_form_name'])
            route_list.append(route_data[0]['route_of_administration_name'])
            brand_namelist.append(filters[filters_ids.index(drug_id)].title())
        
        print("Here are the results that we found:")
        ff_d = {'Brand Name':brand_namelist, 'Form': form_list, 'Administration Route': route_list}
        ff = pd.DataFrame(ff_d)
        return(ff)

# After reading response.json

In [1]:
import pandas as pd
import requests
from pprint import pprint
import requests
import time
import json

def jdrugconsumption(user_input, user_key):
    
    # credentials
    key = user_key
    Cookie = "38771cce9ac0f974522b34af498bfe66=56e404caec9315398b6da15a1e47abfc"

    url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/drugproduct?lang=en&type=json&status=1&user-key="
    url += key

    headers = {
      'user-key': key,
      'Cookie': Cookie
    }
    
    # reading json response from drugproduct API call
    data = pd.read_json("response.json")
    drug_codes = []
    brand_names = []
    code_brands = []

    # converting response to dictionary and creating a lsit of drug_codes and brand_names
    datadict = data.to_dict(orient='records')
    datadict
    n = len(datadict)
    # print( (datadict[7]['drug_code'], datadict[7]['brand_name'] ) )
    for i in range(n):
        drug_codes.append(datadict[i]['drug_code'])
        brand_names.append(datadict[i]['brand_name'])
        code_brands.append((datadict[i]['drug_code'],datadict[i]['brand_name']))
    df = pd.DataFrame({'drug_code': drug_codes, 'brand_name': brand_names})

    user_input = user_input.lower()
    filters = []

    # finding matching brand names for user_input
    for brand in brand_names:
        if user_input in brand.lower():
            filters.append(brand)
            
    # removing duplicates
    filters = list(set(filters)) 

    # finding drug id(s) corresponding to user-input
    filters_ids = []
    for item in filters:
        k = brand_names.index(item)
        filters_ids.append(drug_codes[k])

    #############################################################################################################
    # this code chunk is for testing purposes, please ignore
    # to check drug ids and brand names that are filtered
    # pprint(filters)
    # pprint(filters_ids)
    # print(filters)

    #############################################################################################################

    form_list = []
    route_list = []
    brand_namelist=[]

    # creating urls to get response from form and route APIs
    route_url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/route?lang=en&type=json&id="
    form_url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/form?lang=en&type=json&id="

    # inputting drug ids and retrieving results
    for drug_id in filters_ids:

        # creating url specific to a drug
        route_url += (str(drug_id) + "&active=yes")
        form_url += (str(drug_id) + "&active=yes")

        # getting responses
        route_response = requests.request("GET", route_url, headers=headers)
        time.sleep(1)
        form_response = requests.request("GET", form_url, headers=headers)

        # converting to list of dictionaries using .json()
        route_data = route_response.json()
        form_data = form_response.json()
        
        # extracting required information
        form_list.append(form_data[0]['pharmaceutical_form_name'])
        route_list.append(route_data[0]['route_of_administration_name'])
        brand_namelist.append(filters[filters_ids.index(drug_id)].title())

    # printing/returning output 
    print("Here are the results that we found:")
    ff_d = {'Brand Name':brand_namelist, 'Form': form_list, 'Administration Route': route_list}
    ff = pd.DataFrame(ff_d)
    return (ff)

In [3]:
# 14ec6574be9f7ef563bce7b554798152
jdrugconsumption("PLACIDYL CAP 200MG", "14ec6574be9f7ef563bce7b554798152")

Here are the results that we found:


,Brand Name,Form,Administration Route
0,Placidyl Cap 200Mg,Capsule,Oral


# Code with separate functions (in case coverage is low, implement this)

# find_ids()

In [1]:
import pandas as pd
import requests
from pprint import pprint
import requests
import time
import json

def find_ids(user_input):
    data = pd.read_json("response.json")
    drug_codes = []
    brand_names = []
    code_brands = []

    datadict = data.to_dict(orient='records')
    datadict
    # print( (datadict[7]['drug_code'], datadict[7]['brand_name'] ) )
    n = len(datadict)
    for i in range(n):
        drug_codes.append(datadict[i]['drug_code'])
        brand_names.append(datadict[i]['brand_name'])
        code_brands.append((datadict[i]['drug_code'],datadict[i]['brand_name']))

    df = pd.DataFrame({'drug_code': drug_codes, 'brand_name': brand_names})

#     user_input = "PLACIDYL CAP 200MG" #####################################################################
    user_input = user_input.lower()
    filters = []

    # finding matching brand names
    for brand in brand_names:
        if user_input == brand.lower():
    #         print(brand_name.index(brand))
            filters.append(brand)

    filters = list(set(filters)) # removing duplicates

    # finding corresponding drug ids
    filters_ids = []
    for item in filters:
        k = brand_names.index(item)
        filters_ids.append(drug_codes[k])

    return {"filters_ids": filters_ids, "filters": filters}

In [2]:
# user_input = "PLACIDYL CAP 200MG"
# find_ids(user_input)["filters"]

# find_routeform()

In [3]:
import pandas as pd
import requests
from pprint import pprint
import requests
import time
import json


def routeform(user_input):
    form_list = []
    route_list = []
    brand_namelist=[]
    
    # credentials
    key = "14ec6574be9f7ef563bce7b554798152"
    Cookie = "38771cce9ac0f974522b34af498bfe66=56e404caec9315398b6da15a1e47abfc"

    url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/drugproduct?lang=en&type=json&status=1&user-key="
    url += key

    headers = {
      'user-key': key,
      'Cookie': Cookie
    }

    # creating urls
    route_url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/route?lang=en&type=json&id="
    form_url = "https://dpd-hc-sc-apicast-production.api.canada.ca/v1/form?lang=en&type=json&id="

    myfilters = find_ids(user_input)

    # inputting drug ids and retrieving results
    for drug_id in myfilters["filters_ids"]:

        # creating url
        route_url += (str(drug_id) + "&active=yes")
        form_url += (str(drug_id) + "&active=yes")

        # getting responses
        route_response = requests.request("GET", route_url, headers=headers)
        time.sleep(1)
        form_response = requests.request("GET", form_url, headers=headers)

        # converting to list of dictionaries
        route_data = route_response.json()
        form_data = form_response.json()
        # extracting required information

        form_list.append(form_data[0]['pharmaceutical_form_name'])
        route_list.append(route_data[0]['route_of_administration_name'])
        brand_namelist.append(myfilters["filters"][myfilters["filters_ids"].index(drug_id)].title())

    ff_d = {'Brand Name':brand_namelist, 'Form': form_list, 'Administration Route': route_list}
    ff = pd.DataFrame(ff_d)
    return(ff)

In [ ]:
routeform("LIP BALM SPF 15")
# routeform("ISOPTO CARBACHOL 3%")
# routeform("LIP BALM SPF 15")
# routeform("CHEW C BERRY TABLETS - 125MG")
# routeform("HENRY SCHEIN MINUTE TOPICAL FOAM FLUORIDE")